## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gornozavodsk,RU,2020-08-23 17:20:24,58.38,58.32,49.48,93,17,3.22,few clouds
1,1,Havelock,US,2020-08-23 17:18:39,34.88,-76.90,86.00,62,20,6.93,few clouds
2,2,Kodiak,US,2020-08-23 17:19:34,57.79,-152.41,55.40,87,90,3.36,overcast clouds
3,3,Najran,SA,2020-08-23 17:20:05,17.49,44.13,96.80,16,100,6.93,overcast clouds
4,4,Acapulco,MX,2020-08-23 17:17:16,16.86,-99.89,82.40,83,75,3.36,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Gornozavodsk,RU,2020-08-23 17:20:24,58.38,58.32,49.48,93,17,3.22,few clouds
2,2,Kodiak,US,2020-08-23 17:19:34,57.79,-152.41,55.40,87,90,3.36,overcast clouds
9,9,Dikson,RU,2020-08-23 17:19:23,73.51,80.55,46.54,78,0,9.64,clear sky
10,10,Nikolskoye,RU,2020-08-23 17:19:13,59.70,30.79,66.20,72,90,8.95,overcast clouds
11,11,Rikitea,PF,2020-08-23 17:19:14,-23.12,-134.97,69.44,83,84,22.28,broken clouds
12,12,Talnakh,RU,2020-08-23 17:19:27,69.49,88.40,57.20,93,100,6.71,overcast clouds
14,14,Bredasdorp,ZA,2020-08-23 17:19:36,-34.53,20.04,59.00,82,100,4.70,overcast clouds
17,17,Kaitangata,NZ,2020-08-23 17:19:19,-46.28,169.85,53.01,89,98,15.01,light rain
19,19,Busselton,AU,2020-08-23 17:19:15,-33.65,115.33,45.00,76,1,4.83,clear sky
21,21,Bayan,CN,2020-08-23 17:20:28,46.08,127.40,63.54,88,100,5.30,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                347
City                   347
Country                347
Date                   347
Lat                    347
Lng                    347
Max Temp               347
Humidity               347
Cloudiness             347
Wind Speed             347
Current Description    347
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gornozavodsk,RU,49.48,few clouds,58.38,58.32,
2,Kodiak,US,55.40,overcast clouds,57.79,-152.41,
9,Dikson,RU,46.54,clear sky,73.51,80.55,
10,Nikolskoye,RU,66.20,overcast clouds,59.70,30.79,
11,Rikitea,PF,69.44,broken clouds,-23.12,-134.97,
12,Talnakh,RU,57.20,overcast clouds,69.49,88.40,
14,Bredasdorp,ZA,59.00,overcast clouds,-34.53,20.04,
17,Kaitangata,NZ,53.01,light rain,-46.28,169.85,
19,Busselton,AU,45.00,clear sky,-33.65,115.33,
21,Bayan,CN,63.54,overcast clouds,46.08,127.40,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Gornozavodsk,RU,49.48,few clouds,58.38,58.32,Hotel Kvarotel
2,Kodiak,US,55.40,overcast clouds,57.79,-152.41,Best Western Kodiak Inn and Convention Center
9,Dikson,RU,46.54,clear sky,73.51,80.55,
10,Nikolskoye,RU,66.20,overcast clouds,59.70,30.79,Tourist House - Sablino
11,Rikitea,PF,69.44,broken clouds,-23.12,-134.97,Pension Maro'i


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))